In [2]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import get_gene_and_cohort_list_utils
import configuration_util

filePathDict = configuration_util.get_all_files_path_dict()

In [30]:
def count_mutations_by_dep_map_score(maf, geneSizeDict, scoreDicts):
    
    tsgs = get_gene_and_cohort_list_utils.get_tsgs()
    impactGenes = get_gene_and_cohort_list_utils.get_im6_genes()
    oncogenes = impactGenes - tsgs
    
    truncatingConsequences = set(['Frame_Shift_Del', 'Frame_Shift_Ins', 'Nonsense_Mutation'])
    
    listOfDicts = []
    for cancerType, d in scoreDicts.items():
        print cancerType
        
        cTypeMaf = maf[maf['cancerTypeDepMap'] == cancerType]
        truncatingMaf = cTypeMaf[cTypeMaf['Variant_Classification'].isin(truncatingConsequences)]
        missenseMaf = cTypeMaf[cTypeMaf['Variant_Classification'] == 'Missense_Mutation']
        silentMaf = cTypeMaf[cTypeMaf['Variant_Classification'] == 'Silent']
    
        cntr = 0
        for gene, score in d.items():
        
            if cntr%1000==0:print round((1.0*cntr)/len(set(allExomeHypermutantMaf['Hugo_Symbol'])), 2),
            geneMafTruncating = truncatingMaf[truncatingMaf['Hugo_Symbol'] == gene]
            geneMafMissense = missenseMaf[missenseMaf['Hugo_Symbol'] == gene]
            geneMafSilent = silentMaf[silentMaf['Hugo_Symbol'] == gene]

            nDoubleTruncating = sum([1  if case > 1 else 0 for case, count in dict(geneMafTruncating['Tumor_Sample_Barcode'].value_counts()).items()])
            nDoubleMissense = sum([1 if case > 1 else 0 for case, count in dict(geneMafMissense['Tumor_Sample_Barcode'].value_counts()).items()])
            nDoubleSilent = sum([1 if case > 1 else 0 for case, count in dict(geneMafSilent['Tumor_Sample_Barcode'].value_counts()).items()])

            listOfDicts.append({'gene': gene, 'score': score, 'cancerType': cancerType, 'context': cancerType + '_' + gene,
                                'geneSize': geneSizeDict[gene] if gene in geneSizeDict else None,
                                'nDoubleTruncating': nDoubleTruncating,
                                'nDoubleMissense': nDoubleMissense,
                                'nDoubleSilent': nDoubleSilent, 
                                'nMissense': geneMafMissense.shape[0],
                                'nSilent': geneMafSilent.shape[0],
                                'nTruncating': geneMafTruncating.shape[0]
            })
        
            cntr += 1
    
    return pd.DataFrame(listOfDicts)
    
    
    
    

In [4]:
allExomeHypermutantMaf = pd.read_table(filePathDict['ALL_EXOME_HYPERMUTATOR_MAF'])


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,12,17,19,20,22,25,26,27,29,30,31,32,35,36,50,51,52,54,55,56,57,58,65,66,68,69,73,74,75,82,84,85,89,90,91,92,93,94,95,96,97,101,102,104,106,110,111,112,113,117,118,120,123,124,125,128,135,136,137,138,144,147,150,151,154,155,158,160,161,190,192,200,212,213,214,215,218,219,235,236,238,239,240,242,280,283,286,287,306,307,308,309,312,314,315) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#dictionary to reassign cancer type information

#adjust dep map names to merge cancer types

tcgaToDepMapCancerTypeMapping = {'TCGA-COAD': 'Colon/Colorectal Cancer',
                                'TCGA-READ': 'Colon/Colorectal Cancer',
                                'TCGA-UCEC': 'Endometrial/Uterine Cancer',
                                'TCGA-GBM': 'Brain Cancer',
                                'TCGA-LGG': 'Brain Cancer',
                                'TCGA-LUAD': 'Lung Cancer',
                                'TCGA-LUSC': 'Lung Cancer',
                                'TCGA-SKCM': 'Skin Cancer',
                                'TCGA-BLCA': 'Bladder Cancer',
                                'TCGA-BRCA': 'Breast Cancer',
                                'TCGA-PRAD': 'Prostate Cancer',
                                'TCGA-ESCA': 'Esophageal Cancer',
                                'TCGA-STAD': 'Gastric Cancer'}

impactToDepMapCancerTypeMapping = {
    'Colorectal Cancer': 'Colon/Colorectal Cancer',
    'Endometrial Cancer': 'Endometrial/Uterine Cancer',
    'Glioma': 'Brain Cancer',
    'Non-Small Cell Lung Cancer': 'Lung Cancer',
    'Small Cell Lung Cancer': 'Lung Cancer',
    'Melanoma': 'Skin Cancer',
    'Bladder Cancer': 'Bladder Cancer',
    'Breast Cancer': 'Breast Cancer',
    'Prostate Cancer': 'Prostate Cancer',
    'Esophagogastric Cancer': 'Esophageal Cancer', #change to combine esophagus and gastric
}


tcgaCancerTypeDict = get_gene_and_cohort_list_utils.get_tcga_cancer_type_info(tcgaInfoPath = filePathDict['TCGA_CANCER_TYPE_INFO'], cancerTypes = [])
impactCancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath = filePathDict['CANCER_TYPE_INFO'])


/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:193: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  tcgaDf = pd.read_table(tcgaInfoPath)
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:197: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  impactCancerTypeDf = pd.read_table(impactCancerTypeInfoPath)


In [6]:
exomeIdMapping = get_gene_and_cohort_list_utils.get_exome_id_mapping(exomeIdMappingPath = '~/Desktop/hypermutationProjectFinal/files/infoFiles/dmpToSampleNameMapping.tsv')[0]
allExomeHypermutantMaf['Tumor_Sample_Barcode'] = allExomeHypermutantMaf['Tumor_Sample_Barcode'].apply(lambda x: exomeIdMapping[x] if x in exomeIdMapping else x)


/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:258: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  df = pd.read_table(exomeIdMappingPath)


In [7]:
allExomeHypermutantMaf['Tumor_Sample_Barcode'] = allExomeHypermutantMaf['Tumor_Sample_Barcode'].apply(lambda x:
                        x[:12] if len(x) == 15
                       else x)

allExomeHypermutantMaf['cancerType'] = allExomeHypermutantMaf['Tumor_Sample_Barcode'].apply(lambda x: 
    tcgaCancerTypeDict[x] if x in tcgaCancerTypeDict
    else impactCancerTypeDict[x] if x in impactCancerTypeDict
    else None)


In [8]:
allExomeHypermutantMaf['cancerTypeDepMap'] = allExomeHypermutantMaf.apply(lambda row:
        impactToDepMapCancerTypeMapping[row['cancerType']]
            if row['cohort'] in ['mmr_RECAPTURE', 'pole_RECAPTURE'] and row['cancerType'] in impactToDepMapCancerTypeMapping
        else tcgaToDepMapCancerTypeMapping[row['cancerType']]
            if row['cohort'] in ['mmr_TCGA', 'pole_TCGA'] and row['cancerType'] in tcgaToDepMapCancerTypeMapping
        else 'other', axis=1)

In [9]:
cTypes = set(allExomeHypermutantMaf['cancerTypeDepMap']) - set(['other'])
reload(get_gene_and_cohort_list_utils)
d = get_gene_and_cohort_list_utils.get_cancer_type_specific_dep_map_data(cancerTypes = cTypes)

analyzing  Brain Cancer
analyzing  Prostate Cancer
analyzing  Bladder Cancer
analyzing  Gastric Cancer
analyzing  Esophageal Cancer
analyzing  Skin Cancer
analyzing  Lung Cancer
analyzing  Breast Cancer
analyzing  Endometrial/Uterine Cancer
analyzing  Colon/Colorectal Cancer


In [10]:
allExomeHypermutantMaf = allExomeHypermutantMaf[allExomeHypermutantMaf['cancerTypeDepMap'].isin(cTypes)]
allExomeHypermutantMaf['depMapScore'] = allExomeHypermutantMaf.apply(lambda row:
         d[row['cancerTypeDepMap']][row['Hugo_Symbol']] if row['Hugo_Symbol'] in d[row['cancerTypeDepMap']] else None                                                          
        ,axis=1)

In [11]:
reload(get_gene_and_cohort_list_utils)
geneSizeDict = get_gene_and_cohort_list_utils.get_gene_size_info()

/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:32: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  geneLengthInfo = pd.read_table('~/Desktop/hypermutationProjectFinal/files/infoFiles/all_gene_coding_sequences.tsv')


In [13]:
allExomeHypermutantMaf['geneSize'] = allExomeHypermutantMaf['Hugo_Symbol'].apply(lambda x:
        geneSizeDict[x] if x in geneSizeDict else None)

In [31]:
df = count_mutations_by_dep_map_score(allExomeHypermutantMaf, geneSizeDict, d)

Brain Cancer
0.0 0.0468823253633 0.0937646507267 0.14064697609 0.187529301453 0.234411626817 0.28129395218 0.328176277543 0.375058602907 0.42194092827 0.468823253633 0.515705578997 0.56258790436 0.609470229723 0.656352555087 0.70323488045 0.750117205813 0.796999531177 0.84388185654 Prostate Cancer
0.0 0.0468823253633 0.0937646507267 0.14064697609 0.187529301453 0.234411626817 0.28129395218 0.328176277543 0.375058602907 0.42194092827 0.468823253633 0.515705578997 0.56258790436 0.609470229723 0.656352555087 0.70323488045 0.750117205813 0.796999531177 0.84388185654 Bladder Cancer
0.0 0.0468823253633 0.0937646507267 0.14064697609 0.187529301453 0.234411626817 0.28129395218 0.328176277543 0.375058602907 0.42194092827 0.468823253633 0.515705578997 0.56258790436 0.609470229723 0.656352555087 0.70323488045 0.750117205813 0.796999531177 0.84388185654 Gastric Cancer
0.0 0.0468823253633 0.0937646507267 0.14064697609 0.187529301453 0.234411626817 0.28129395218 0.328176277543 0.375058602907 0.42194

In [37]:
nCasesDict = {}
for cType in set(df['cancerType']):
    nCases = len(set(allExomeHypermutantMaf[allExomeHypermutantMaf['cancerTypeDepMap'] == cType]['Tumor_Sample_Barcode']))
    nCasesDict[cType] = nCases
    
df['nCancerType'] = df['cancerType'].apply(lambda x: nCasesDict[x])


In [39]:
df.to_csv('~/Desktop/WORK/dataForLocalPlotting/essentialGeneInfo.tsv', index=False, sep='\t')

In [42]:
df.columns.values

array(['cancerType', 'context', 'gene', 'geneSize', 'nDoubleMissense',
       'nDoubleSilent', 'nDoubleTruncating', 'nMissense', 'nSilent',
       'nTruncating', 'score', 'nCancerType'], dtype=object)